In [1]:
from Bio import Entrez

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
def search(query):
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='1000',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [5]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [6]:
search_word='fuel cell'

In [7]:
results = search(search_word)
id_list = results['IdList']
papers = fetch_details(id_list)
for i, paper in enumerate(papers['PubmedArticle']):
    print("%d) %s" % (i+1, paper['MedlineCitation']['Article']['ArticleTitle']))

1) Increased power generation from cylindrical microbial fuel cell inoculated with P. aeruginosa.
2) Synthesis of nickel-based layered double hydroxide (LDH) and their adsorption on carbon felt fibres: application as low cost cathode catalyst in microbial fuel cell (MFC).
3) Microbial fuel cells: From fundamentals to applications. A review.
4) Geoelectrodes and Fuel Cells for Simulating Hydrothermal Vent Environments.
5) Highly durable, coking and sulfur tolerant, fuel-flexible protonic ceramic fuel cells.
6) Noble Metal Based Alloy Nanoframes: Syntheses and Applications in Fuel Cells.
7) Investigation of Perovskite Oxide SrCo<sub>0.8</sub> Cu<sub>0.1</sub> Nb<sub>0.1</sub> O<sub>3-δ</sub> as a Cathode Material for Room Temperature Direct Ammonia Fuel Cells.
8) Fuel Production from Seawater and Fuel Cells Using Seawater.
9) Hypophosphites as Eco-Compatible Fuels for Membrane-Free Direct Liquid Fuel Cells.
10) Role of dissolved oxygen on the degradation mechanism of Reactive Green 19 an

In [8]:
papers.keys()

dict_keys(['PubmedArticle', 'PubmedBookArticle'])

In [9]:
papers['PubmedArticle'][0].keys()

dict_keys(['MedlineCitation', 'PubmedData'])

In [10]:
papers['PubmedArticle'][0]['MedlineCitation'].keys()

dict_keys(['KeywordList', 'GeneralNote', 'CitationSubset', 'SpaceFlightMission', 'OtherAbstract', 'OtherID', 'PMID', 'DateRevised', 'Article', 'MedlineJournalInfo'])

# Key Word List

In [11]:
[str(i) for i in papers['PubmedArticle'][0]['MedlineCitation']['KeywordList'][0]]

['Carbon brush cylindrical microbial fuel cell',
 'Internal resistance',
 'Microbial fuel cell',
 'P. aeruginosa',
 'Polarization curve',
 'Power overshoot']

# Author List

In [12]:
papers['PubmedArticle'][0]['MedlineCitation']['Article'].keys()

dict_keys(['ELocationID', 'Language', 'ArticleDate', 'Journal', 'ArticleTitle', 'Pagination', 'Abstract', 'AuthorList', 'PublicationTypeList'])

In [13]:
papers['PubmedArticle'][1]['MedlineCitation']['Article']['AuthorList']

ListElement([DictElement({'Identifier': [StringElement('https://orcid.org/0000-0003-0505-7164', attributes={'Source': 'ORCID'})], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': "a Université des Sciences et de la Technologie d'Oran Mohamed-Boudiaf , Oran , Algérie."}, {'Identifier': [], 'Affiliation': "b Laboratoire d'Ingénierie des Procédés de l'Environnement, Université des Sciences et de la Technologie d'Oran Mohamed-Boudiaf, Oran, Algérie."}], 'LastName': 'Djellali', 'ForeName': 'Meriem', 'Initials': 'M'}, attributes={'ValidYN': 'Y'}), DictElement({'Identifier': [], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': "a Université des Sciences et de la Technologie d'Oran Mohamed-Boudiaf , Oran , Algérie."}, {'Identifier': [], 'Affiliation': "b Laboratoire d'Ingénierie des Procédés de l'Environnement, Université des Sciences et de la Technologie d'Oran Mohamed-Boudiaf, Oran, Algérie."}], 'LastName': 'Kameche', 'ForeName': 'Mostefa', 'Initials': 'M'}, attributes={'ValidYN': 

In [14]:
paper_author_lst=[i['MedlineCitation']['Article']['AuthorList'] for i in papers['PubmedArticle']]

In [15]:
len(paper_author_lst)

1000

In [16]:
dfs=[pd.DataFrame(paper_author_lst[i]) for i in range(len(paper_author_lst))]

In [17]:
names_dfs=pd.concat(dfs, axis=0, sort=True )

In [18]:
names_dfs.head()

,AffiliationInfo,ForeName,Identifier,Initials,LastName,Suffix
0,"[{'Identifier': [], 'Affiliation': 'State Key ...",Man,[],M,Zhang,NaN
1,"[{'Identifier': [], 'Affiliation': 'State Key ...",Zhaokun,[],Z,Ma,NaN
2,"[{'Identifier': [], 'Affiliation': 'State Key ...",Na,[],N,Zhao,NaN
3,"[{'Identifier': [], 'Affiliation': 'State Key ...",Kaixuan,[],K,Zhang,NaN
4,"[{'Identifier': [], 'Affiliation': 'State Key ...",Huaihe,[],H,Song,NaN


In [19]:
author_count_df=names_dfs[['ForeName', 'LastName']].groupby(['ForeName', 'LastName']).size().reset_index(name='count').sort_values(by='count', ascending=False)

In [20]:
author_count_df.hist();

# Top Authors

In [21]:
top=author_count_df.head(10)

In [22]:
top

,ForeName,LastName,count
375,Carlo,Santoro,19
2667,Plamen,Atanassov,16
94,Alexey,Serov,16
1168,Hong,Liu,13
1306,Ioannis,Ieropoulos,13
1562,John,Greenman,12
3513,Wei,Zhou,9
2908,S K,Kamarudin,8
2931,San Ping,Jiang,8
317,Booki,Min,8


In [23]:
google_url='https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q='

In [24]:
name=top['ForeName']+' '+top['LastName']

In [25]:
result=top.reset_index(drop=True).join(pd.DataFrame({'Google Scholar':[google_url+i for i in name.str.replace(' ', '+')+'+review&oq=']}))

In [26]:
pd.set_option('display.max_colwidth', 1000)

In [27]:
result

,ForeName,LastName,count,Google Scholar
0,Carlo,Santoro,19,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Carlo+Santoro+review&oq=
1,Plamen,Atanassov,16,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Plamen+Atanassov+review&oq=
2,Alexey,Serov,16,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Alexey+Serov+review&oq=
3,Hong,Liu,13,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hong+Liu+review&oq=
4,Ioannis,Ieropoulos,13,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Ioannis+Ieropoulos+review&oq=
5,John,Greenman,12,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=John+Greenman+review&oq=
6,Wei,Zhou,9,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wei+Zhou+review&oq=
7,S K,Kamarudin,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=S+K+Kamarudin+review&oq=
8,San Ping,Jiang,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=San+Ping+Jiang+review&oq=
9,Booki,Min,8,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Booki+Min+review&oq=


In [28]:
print(pd.DataFrame({'Google Scholar':[google_url+i for i in name.str.replace(' ', '+')+'+review&oq=']}))

                                                                            Google Scholar
0       https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Carlo+Santoro+review&oq=
1    https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Plamen+Atanassov+review&oq=
2        https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Alexey+Serov+review&oq=
3            https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hong+Liu+review&oq=
4  https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Ioannis+Ieropoulos+review&oq=
5       https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=John+Greenman+review&oq=
6            https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wei+Zhou+review&oq=
7       https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=S+K+Kamarudin+review&oq=
8      https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=San+Ping+Jiang+review&oq=
9           https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Booki+Min+review&oq=

# Key Words from Title

In [29]:
titles=[i['MedlineCitation']['Article']['ArticleTitle'].lower().replace(',','').replace('.','').replace(':', '').replace('?','').replace('<sub>', '').replace('</sub>','').replace('<i>','').replace('</i>','').replace(search_word,'') for i in papers['PubmedArticle']]

In [32]:
tfidf=TfidfVectorizer(ngram_range=(2,2),stop_words='english')

In [34]:
X=tfidf.fit_transform(titles)

In [39]:
tfidf_df=pd.DataFrame(X.todense(), columns=sorted(tfidf.vocabulary_))

In [40]:
key_rank=tfidf_df.sum().sort_values(ascending=False)

In [41]:
key_rank[:20]

solid oxide                  14.399595
exchange membrane            10.642937
electricity generation       10.300425
proton exchange               9.668957
polymer electrolyte           8.670652
oxygen reduction              7.216754
microbial community           6.840843
performance microbial         6.668812
wastewater treatment          6.550377
single chamber                6.477291
direct methanol               6.390863
air cathode                   6.128711
chamber microbial             6.057837
high performance              5.963384
generation microbial          5.749137
temperature solid             5.208095
electrolyte membrane          5.097043
bioelectricity generation     4.863232
cathode microbial             4.855944
electricity production        4.520554
dtype: float64

In [42]:
key_rank[(key_rank>key_rank.quantile(.75))&(key_rank<key_rank.quantile(.8))][:20]

characterization electricity    0.408083
structure long                  0.408083
phototrophic bacteria           0.408021
using anoxygenic                0.408021
insights photosynthetic         0.408021
anoxygenic phototrophic         0.408021
validation chromium             0.407935
chromium getters                0.407935
getters solid                   0.407935
development validation          0.407935
applications overview           0.407465
based nanohybrid                0.407465
nanohybrid proton               0.407465
study ceria                     0.406627
nanocomposite electrolytes      0.406627
carbonate nanocomposite         0.406627
ceria carbonate                 0.406627
temperature pem                 0.405939
membranes high                  0.405939
electrolysers critical          0.405939
dtype: float64

LDA
word Vec
pyldavis

pyldavis

textrank
lexrank